In [12]:
import pandas as pd 

In [13]:
# Reading in new dataset create through combining Kaggle dataset with TMDb API information 
df = pd.read_csv("movies_with_tmdb_data_complete.csv")

In [14]:
# First few rows of the dataframe
df.head()

,Unnamed: 0,ID,Title,Year,Age,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,title,tmdb_id,budget,revenue,runtime,status
0,0,1,The Irishman,2019,18+,98/100,1,0,0,0,0,The Irishman,398978.0,159000000.0,968853.0,209.0,success
1,1,2,Dangal,2016,7+,97/100,1,0,0,0,0,Dangal,360814.0,10400000.0,311000000.0,161.0,success
2,2,3,David Attenborough: A Life on Our Planet,2020,7+,95/100,1,0,0,0,0,David Attenborough: A Life on Our Planet,664280.0,0.0,0.0,83.0,success
3,3,4,Lagaan: Once Upon a Time in India,2001,7+,94/100,1,0,0,0,0,Lagaan: Once Upon a Time in India,19666.0,5200000.0,8100000.0,233.0,success
4,4,5,Roma,2018,18+,94/100,1,0,0,0,0,Roma,426426.0,15000000.0,1140769.0,135.0,success


In [18]:
import urllib
baseurl="https://api.nytimes.com/svc/search/v2/articlesearch.json?" 

In [19]:
import json
import time
from urllib.error import HTTPError

## Running Using All API Keys at Once

In [21]:
# category_counts = df['Year'].value_counts()
# category_counts

In [ ]:
import pandas as pd

# Using all 4 API keys (all group members)
api_keys = [
    "kCIUaGCa2xAl7zYGN5EaRegeKwH6mLFX",
    "bkjWbL5kPAjxqKS00mhPbtUJmhhJgJZc", 
    "Da0kHVMzO4ChQxTb5AwiJcxa22wIJ4OO",
    "drdQMllid3wAYKAgIRYlZPALtmurp0pF"
]

# get frequency of articles for each movie in the release year 
movie_hits = [] # initialize empty list to store results
movie_title = []

# Use .iterrows() in pandas package to iterate through the dataset row by row and return tuples of (index, row info)
for idx, row in titles.iterrows(): # idx is the index (row)
    # iterates through the row numbers of the dataset 

    # Getting movie title and release year for the row 
    title = row["Title"]
    year = int(row["Year"])

    # Rotate through the 4 API keys using modulo operator to prevent hiting limit
    current_api_key = api_keys[idx % 4] # gives the remainder of the index divided by 4 

    pa = {
        "api-key": current_api_key, 
        "q": title,
        "begin_date": f"{year}0101",  # hits in the release year
        "end_date": f"{year}1231",
        "fq": 'section.name:("Movies")' # filtering for movie section 
    }

    url2check = baseurl + urllib.parse.urlencode(pa)
    # send the request and get response
    while True:
        try:
            result = urllib.request.urlopen(url2check).read()
            resd = json.loads(result)
            hit = resd['response']['metadata']['hits']
            movie_hits.append(hit)
            movie_title.append(title)
            print(f"Successfully retrieved {hit} hits for {title} at index {idx}")
            time.sleep(7)
            break  # exit while True, go to next movie
        except HTTPError as e:
            print(f"HTTP Error {e.code} for index {idx}: {e.reason}")
            if e.code == 429:
                print("Rate limit exceeded. Waiting longer before retrying...")
                time.sleep(60)
                # loop continues and retries same movie
            else:
                # for other errors, maybe store None and move on
                movie_hits.append(None)
                movie_title.append(title)
                break
                
    time.sleep(7) # wait 7 seconds between each request

# Add the hits to original df
titles['Movie_Hits'] = movie_hits

# Save back to CSV
titles.to_csv('final_movies_enriched.csv', index=False)

In [ ]:
# Fix to error from last code 
titles.loc[:, 'Movie_Hits'] = movie_hits
titles.to_csv('final_movies_enriched.csv', index=False)

In [42]:
# Reading in datasets from NYT and TMDb API
df_hits = pd.read_csv("movies_hits.csv")
df_tmdb = pd.read_csv("movies_with_tmdb_data_complete.csv")

In [43]:
# Checking column names 
print(df_hits.columns)
print(df_tmdb.columns)

Index(['Title', 'Year', 'Movie_Hits'], dtype='object')
Index(['Unnamed: 0', 'ID', 'Title', 'Year', 'Age', 'Rotten Tomatoes',
       'Netflix', 'Hulu', 'Prime Video', 'Disney+', 'Type', 'title', 'tmdb_id',
       'budget', 'revenue', 'runtime', 'status'],
      dtype='object')


In [44]:
# Dropping unnecessary columns 
df_tmdb = df_tmdb.drop(columns=["title"])
df_tmdb = df_tmdb.drop(columns=["Unnamed: 0"])
df_tmdb = df_tmdb.drop(columns=["status"])
df_tmdb = df_tmdb.drop(columns=["Type"])

In [45]:
# Merging the dataframes on movie title and release year 
merged = df_hits.merge(df_tmdb, on=["Title", "Year"], how="outer")
merged.to_csv("movies_final.csv", index=False)